# <font color='black'>Регрессионный анализ: продолжение, 2024 </font>
## <font color='black'> Гетероскедастичность </font>
В рамках данного практического занятия мы продолжим работать с данными из статьи [Kalenborn C., Lessman C., 2013](https://yadi.sk/i/nlEQUoWKiqY0UA). Одна из частей анализа в данной статье выполнена на основе cross-section data (использованы усредненные данные за 2005 - 2010 гг.). Возьмем такой массив, так как мы пока не знакомились с моделями для анализа панельных данных.

Стоит отметить, что авторы изучают взаимосвязь уровня коррупции (является откликом в регрессионной модели) и демократии, предполагая, что ее характер зависит от значений показателя свободы прессы. Мы протестируем данную гипотезу на практическом занятии 2, когда познакомимся с регрессионными моделями с переменными взаимодействия. Кратко о данных:
* cpi - уровень коррупции: Corruption Perception Index. Приведен к непрерывной шкале от 0 до 10, где 10 означает наиболее высокий уровень коррупции.
* dem - индекс демократии: Vanhanen’s democratization index. Непрерывная шкала от 0 до 100, где 100 означает максимальное значение уровня демократии.
* fp - свобода прессы: Freedom House. Приведен к непрерывной шкале от 0 до 100, где 100 - наиболее высокое значение свободы прессы.
* loggdppc - натуральный логарифм ВВП на душу населения: World Bank.
* stab - уровень политической стабильности. Индекс построен на основе показателей "Political Stability" и "Absence of Violence/Terrorism" из the Worldwide Governance Indicators. Непрерывная шкала от -2.5 до 2.5, где 2.5 соответствует наиболее высокому уровню политической стабильности.
* britcol - дамми-переменная, где 1 - бывшая британская колония.

In [ ]:
import pandas as pd
import statsmodels as sm
import statsmodels.formula.api as smf
from scipy import stats
from scipy.stats import shapiro
import seaborn as sns
import numpy as np
import matplotlib.pyplot as mpl

from statsmodels.tools.tools import add_constant

from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.diagnostic import het_white
from statsmodels.stats.diagnostic import het_goldfeldquandt

from statsmodels.stats.outliers_influence import outlier_test

Откроем массив данных для репликации результатов исследования: lab1.dta

In [ ]:
lab1 = pd.read_stata('lab1.dta')
lab1 = lab1.dropna()

In [ ]:
m1 = smf.ols(formula = "cpi ~ dem + fp + stab + loggdppc + britcol", data = lab1).fit()
print(m1.summary())

Проверим, можно ли говорить про гомоскедастичность (здравый смысл подсказывает, что надеяться не стоит), но тем не менее, реализуем определенные диагностики. Для начала построим график, показывающий, как связаны остатки в квадрате и одна из объясняющих переменных - к примеру, ключевой предиктор (dem).

In [ ]:
fitted = m1.predict()
residuals_sq = m1.resid**2
fig, ax = mpl.subplots()
sns.scatterplot(x = lab1['dem'], y = residuals_sq)
ax.set_xlabel( "Democracy")
ax.set_ylabel( "Squared residuals")

Можно не рассматривать объясняющие переменные по отдельности, а  построить график, показывающий, как связаны остатки в квадрате и предсказанные значения по модели ($\hat{y}$ как линейная комбинация предикторов)

In [ ]:
fitted = m1.predict()
residuals_sq = m1.resid**2
fig, ax = mpl.subplots()
sns.scatterplot(x = fitted, y = residuals_sq)
ax.set_xlabel( "Fitted values")
ax.set_ylabel( "Squared residuals")

Есть и формальные тесты для проверки гипотезы о гомоскедастичности. К примеру, тест Бреуша-Пагана. Давайте его реализуем и проинтепретируем результаты на основе p-value.

In [ ]:
X = lab1[["dem", "fp", "stab", "loggdppc", "britcol"]]
X = add_constant(X)

white_test = het_white(m1.resid, X)
white_test

stat, p_value = white_test[0:2]
print(f'Statistic: {stat}, P-Value: {p_value}')

In [ ]:
white_test1 = het_white(m1.resid, m1.model.exog)
stat, p_value = white_test1[0:2]
print(f'Statistic: {stat}, P-Value: {p_value}')

Как мы уже обсуждали, можно использовать тест Бреуша-Пагана, предполагающий более экономную спецификацию вспомогательной модели (остатки в квадрате регрессируются на исходные объясняющие переменные, не учитываются квадратичные эффекты и совместные эффекты).

In [ ]:
bp_test = het_breuschpagan(m1.resid, m1.model.exog)
stat, p_value = bp_test[0:2]
print(f'Statistic: {stat}, P-Value: {p_value}')

Кроме этого, мы можем рассмотреть частный случай гетероскедастичности. Мы можем предположить, что вариация остатков монотонно возрастает/убывает с ростом предиктора. Для иллюстрации возьмем предиктор "fp".

Только для начала проверим, можно ли считать распределение остатков нормальным.

In [ ]:
shapiro(m1.resid)

In [ ]:
gq_test = het_goldfeldquandt(lab1.cpi, X, idx = 2, drop = 0.2, alternative = 'two-sided')
stat, p_value, alternative = gq_test[0:3]
print(f'Statistic: {stat}, P-Value: {p_value}, alternative: {alternative}')

Переоценим модель с робастными стандартными ошибками (HC3).

In [ ]:
m1_1 = smf.ols(formula = "cpi ~ dem + fp + stab + loggdppc + britcol", data = lab1).fit(cov_type = "HC3")
print(m1_1.summary())

Проверим, есть ли в нашем массиве нетипичные наблюдения. Для начала рассмотрим нетипичные наблюдения по предикторам.

In [ ]:
leverage = m1_1.get_influence().hat_matrix_diag

leverage_data = lab1[leverage > 2*np.mean(leverage)]
leverage_data

Посмотрим, как обстоят дела с outliers (нетипичные наблюдения по y).

In [ ]:
student_resid = outlier_test(m1_1).iloc[:, 0]
student_resid

outlier_data = lab1[np.abs(student_resid) > 3]
outlier_data

Диагностируем, есть ли влиятельные наблюдения, используя как меру DFBETA (в данном случае для параметра при dem), так и меру Кука.

In [ ]:
dfbetas = m1_1.get_influence().dfbetas

In [ ]:
coef_index = 1
lab1['dfbetas_dem'] = np.abs(dfbetas[:, coef_index])

dfbeta_dem_data = lab1[np.abs(dfbetas[:, coef_index]) > 2 / np.sqrt(len(lab1))]

dfbeta_dem_data_ordered = dfbeta_dem_data.sort_values('dfbetas_dem', ascending = False)
dfbeta_dem_data_ordered


In [ ]:
cooks_dist = m1_1.get_influence().cooks_distance[0]
cooks_dist

lab1['cooks_dist'] = cooks_dist

influential_obs = lab1[cooks_dist > 4 / len(lab1)]

influential_obs_ordered = influential_obs.sort_values('cooks_dist', ascending = False)
influential_obs_ordered